# Verify jupyter setup by grabbing a few scenes and making a movie



In [1]:
!datacube --version  ## Open Data Cube is a Github Project

Open Data Cube core, version 1.6rc1


In [2]:
!datacube product list   ## I built this cube to have three products/collections/dataset (SETS)

description: Landsat 5 US/ARD Product Definition
format: GeoTiff
id: 51
instrument: TM
lat: null
lon: null
name: landsat_5_USARD
platform: LANDSAT_5
product_type: USARD
time: null
---
description: Landsat 7 US/ARD Product Definition
format: GeoTiff
id: 52
instrument: ETM
lat: null
lon: null
name: landsat_7_USARD
platform: LANDSAT_7
product_type: USARD
time: null
---
description: Landsat 8 US/ARD Product Definition
format: GeoTiff
id: 53
instrument: OLI_TIRS
lat: null
lon: null
name: landsat_8_USARD
platform: LANDSAT_8
product_type: USARD
time: null


In [3]:
## aws s3 ls - would tell you the same counts in the bucket ga-odc-usgs-l1til-west h03v03

!datacube dataset search product ='landsat_5_USARD' |grep id |wc -l
!datacube dataset search product ='landsat_7_USARD' |grep id |wc -l
!datacube dataset search product ='landsat_8_USARD' |grep id |wc -l

1143
923
282


# Fliter by cloud cover

- we could also use pixel_qa to do more detailed masking in the xarray below

In [4]:
import datacube
import time

from datetime import datetime

dc = datacube.Datacube()

selected_product = 'landsat_7_USARD'

date_range = (
        datetime(2015,1,1),
        datetime(2015,12,31))
dss = dc.find_datasets(product=selected_product, time=date_range, measurements=['red',])

clear_datasets = []
for item in dss:
    
    if float(item.metadata_doc['cloud_cover']) < 30.0:
        if float(item.metadata_doc['fill']) < 30.0:
            clear_datasets.append(item)
            
print("Number of Pretty Scenes", len(clear_datasets))
for citem in clear_datasets:
    print(citem.uris)

    




Number of Pretty Scenes 7
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20151005/LE07_CU_003003_20151005_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150919/LE07_CU_003003_20150919_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150818/LE07_CU_003003_20150818_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150717/LE07_CU_003003_20150717_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150701/LE07_CU_003003_20150701_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150615/LE07_CU_003003_20150615_20170713_C01_V01.xml']
['s3://ga-odc-eros-l1til-west/til/003003/LE07/20150223/LE07_CU_003003_20150223_20170713_C01_V01.xml']


In [5]:
Start = time.time()

In [6]:
! banner Load Images

#
#         ####     ##    #####
#        #    #   #  #   #    #
#        #    #  #    #  #    #
#        #    #  ######  #    #
#        #    #  #    #  #    #
#######   ####   #    #  #####

  ###
   #     #    #    ##     ####   ######   ####
   #     ##  ##   #  #   #    #  #       #
   #     # ## #  #    #  #       #####    ####
   #     #    #  ######  #  ###  #            #
   #     #    #  #    #  #    #  #       #    #
  ###    #    #  #    #   ####   ######   ####



In [7]:
ds2 = dc.load(product=selected_product, datasets=clear_datasets, measurements=['red','green','blue'], x = (-122.65,-122.75), y =(45.59,45.65),
              output_crs = 'epsg:5072', resolution = (-30,30))

TONY load function
[(-122.65, 45.59), (-122.75, 45.59), (-122.75, 45.65), (-122.65, 45.65), (-122.65, 45.59)]
[(-2060460.0, 2809680.0), (-2060460.0, 2801070.0), (-2051100.0, 2801070.0), (-2051100.0, 2809680.0), (-2060460.0, 2809680.0)]
Tony load_data
{'aliases': ['band_3', 'red'], 'nodata': -9999, 'name': 'red', 'dtype': 'int16', 'units': '1'}
s3://ga-odc-eros-l1til-west/til/003003/LE07/20150223/LE07_CU_003003_20150223_20170713_C01_V01_SRB3.tif
{'aliases': ['band_3', 'red'], 'nodata': -9999, 'name': 'red', 'dtype': 'int16', 'units': '1'}
s3://ga-odc-eros-l1til-west/til/003003/LE07/20150615/LE07_CU_003003_20150615_20170713_C01_V01_SRB3.tif
{'aliases': ['band_3', 'red'], 'nodata': -9999, 'name': 'red', 'dtype': 'int16', 'units': '1'}
s3://ga-odc-eros-l1til-west/til/003003/LE07/20150701/LE07_CU_003003_20150701_20170713_C01_V01_SRB3.tif
{'aliases': ['band_3', 'red'], 'nodata': -9999, 'name': 'red', 'dtype': 'int16', 'units': '1'}
s3://ga-odc-eros-l1til-west/til/003003/LE07/20150717/LE07_CU

In [8]:
End = time.time()

print ("Load time took seconds = ", End-Start)

Load time took seconds =  10.67040205001831


## Imagine - getting your area of interest (AOI) anywhere in the US in just 2 minutes!

In [9]:
ds2.all

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:  (time: 7, x: 312, y: 287)
Coordinates:
  * time     (time) datetime64[ns] 2015-02-23T18:54:51.875520 ...
  * y        (y) float64 2.81e+06 2.81e+06 2.81e+06 2.81e+06 2.81e+06 ...
  * x        (x) float64 -2.06e+06 -2.06e+06 -2.06e+06 -2.06e+06 -2.06e+06 ...
Data variables:
    red      (time, y, x) int16 339 367 367 367 367 339 339 311 367 396 367 ...
    green    (time, y, x) int16 520 520 455 520 487 487 520 520 551 551 520 ...
    blue     (time, y, x) int16 377 377 319 377 319 377 347 347 377 407 347 ...
Attributes:
    crs:      epsg:5072>

In [10]:
from rasterio.plot import show_hist
from matplotlib import pyplot
%matplotlib inline
from utils.utils import threeBandImage
from utils.utils import loadConfigExtent
def plot_me(ds, time, count):
    figsize = [8,8]
    
    img_toshow = threeBandImage(ds,bands = ['red', 'green', 'blue'], time = time)
    fig = pyplot.figure(figsize = figsize)
    pyplot.imshow(img_toshow)
    ax = pyplot.gca()
    title_string = "IMG " + str(count)
    figName = title_string + '.png'
    title_string = title_string + " --> " + str(ds.time[time].values).split('T')[0]
    time_string = str(ds.time[time].values).split('T')[0]
    figName = time_string + '.png'
    ax.set_title(title_string, fontweight = 'bold', fontsize = 16)
    ax.set_xticklabels(ds.x.values)
    ax.set_yticklabels(ds.y.values)
    ax.set_xlabel('Easting', fontweight = 'bold')
    ax.set_ylabel('Northing', fontweight = 'bold')
    pyplot.savefig(figName)
    # if you remove the close below the images will display - uses lots of memory though
    pyplot.close(fig)
    return True

In [11]:
ntime = len(ds2.time)
cnt = 0
for t in range (ntime):
    # print("Hi",cnt)
    cnt = cnt + 1
    plot_me(ds2, t, cnt)

/home/ubuntu/.local/lib/python3.5/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


In [12]:
! banner Make Movie

#     #
##   ##    ##    #    #  ######
# # # #   #  #   #   #   #
#  #  #  #    #  ####    #####
#     #  ######  #  #    #
#     #  #    #  #   #   #
#     #  #    #  #    #  ######

#     #
##   ##   ####   #    #     #    ######
# # # #  #    #  #    #     #    #
#  #  #  #    #  #    #     #    #####
#     #  #    #  #    #     #    #
#     #  #    #   #  #      #    #
#     #   ####     ##       #    ######



In [13]:

! convert -delay 100 *.png -loop 10 l7_movie.gif
#! scp movie.gif sleepy:.

! rm *.png

In [14]:
! date

Thu Aug  9 21:47:36 UTC 2018


In [15]:
! banner send movie to the cloud
! aws s3 cp l7_movie.gif s3://ga-odc-eros-archive-west/


  ####   ######  #    #  #####
 #       #       ##   #  #    #
  ####   #####   # #  #  #    #
      #  #       #  # #  #    #
 #    #  #       #   ##  #    #
  ####   ######  #    #  #####


 #    #   ####   #    #     #    ######
 ##  ##  #    #  #    #     #    #
 # ## #  #    #  #    #     #    #####
 #    #  #    #  #    #     #    #
 #    #  #    #   #  #      #    #
 #    #   ####     ##       #    ######


  #####   ####
    #    #    #
    #    #    #
    #    #    #
    #    #    #
    #     ####


  #####  #    #  ######
    #    #    #  #
    #    ######  #####
    #    #    #  #
    #    #    #  #
    #    #    #  ######


  ####   #        ####   #    #  #####
 #    #  #       #    #  #    #  #    #
 #       #       #    #  #    #  #    #
 #       #       #    #  #    #  #    #
 #    #  #       #    #  #    #  #    #
  ####   ######   ####    ####   #####

upload: ./l7_movie.gif to s3://ga-odc-eros-archive-west/l7_movie.gif


In [16]:
End2 = time.time()
print ("Movie build time in seconds = ", End2 - End)

Movie build time in seconds =  9.31971287727356


![run movie](http://ga-odc-eros-archive-west.s3.amazonaws.com/l7_movie.gif)

In [17]:
! banner "The End"


#######                         #######
   #     #    #  ######         #        #    #  #####
   #     #    #  #              #        ##   #  #    #
   #     ######  #####          #####    # #  #  #    #
   #     #    #  #              #        #  # #  #    #
   #     #    #  #              #        #   ##  #    #
   #     #    #  ######         #######  #    #  #####



In [18]:
! date

Thu Aug  9 21:47:38 UTC 2018
